In [1]:
import pandas as pd
from openai import OpenAI
import json
import deepl

In [2]:
def read_prompt(file_path):
    """
    Reads content from a text file and returns it as a string
    
    Args:
        file_path (str): Path to the text file
        
    Returns:
        str: Content of the text file
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return None

# Example usage:
nonfabricdetect  = "prompt/nonfabricationhallucinationcheckingprompt.txt"
text_content = read_prompt(nonfabricdetect)


In [3]:
auth_key = "c1a527ba-8719-2649-bdb4-3ee5f4258862:fx"
translator = deepl.Translator(auth_key)


def korean_translater(prompt):
    """
    Translates the prompt to Korean
    
    Args:
        prompt (str): Prompt to be translated
        
    Returns:
        str: Translated prompt
    """
    answer = translator.translate_text(prompt, target_lang="KO")
    
    return answer

In [4]:
#Phase 1
#Determine whether the QA is nonfabrication hallucination

question = '피보험자가 동시에 여러 종류의 수술을 받은 경우 수술급여금은 어떻게 지급됩니까?'
answer = '가장 높은 급여에 해당하는 한 종류의 수술에 대해서만 수술급여금을 지급합니다. 단, 동일한 신체부위가 아니며 의학적으로 치료목적이 다른 독립적인 수술인 경우에는 각각의 수술급여금을 지급합니다.'

def nonfabrication_hallucination_checking(question, answer):
    
    context = {
        'question': question,
        'answer': answer
    }
    
    nonfabricdetectprompt = read_prompt('prompt/nonfabricationhallucinationcheckingprompt.txt').format(**context)
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": nonfabricdetectprompt}
        ]
    )
    answer = completion.choices[0].message.content
    return answer
    # if 'NONE' not in answer:
    #     print("Phase 1 passed")
    #     return answer
    # else:
    #     print("질문과 관련이 없는 답변을 하는 할루시네이션")
    #     return answer
    

nonfabrication_hallucination_checking(question, answer)

'The question asks how the surgical benefits are paid when the insured has undergone multiple types of surgery simultaneously. The answer specifies that the benefit will be paid only for the one type of surgery that corresponds to the highest benefit amount. It also notes that if the surgeries are independent, with different medical treatment purposes and not on the same body part, each surgical benefit will be paid. Therefore, the corresponding specific payment policy detailed in the answer is: "The benefit is paid only for the one type of surgery that corresponds to the highest benefit amount, unless the surgeries are independent and on different body parts."'

In [5]:
# #Phase 2-1 Step-wise Reasoning and Query Decomposition

# def stepwise_reasoning_and_query_decomposition(question, answer):
#     context = {
#         'question': question,
#         'answer': answer
#     }
    
#     querydecomposition = read_prompt('prompt/QueryDecomposition.txt').format(**context)
#     client = OpenAI()
#     completion = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "user", "content": querydecomposition}
#         ]
#     )
#     answer = completion.choices[0].message.content
    
#     return answer


# decomposition = stepwise_reasoning_and_query_decomposition(question, answer)
# print(decomposition)


#해당 방법은 RAG를 통해 이루어진 QA에 대해서 질문을 분해하기에느 적합하지 않기 때문에 사용하지 않는다.
#다만, 답변을 생성할 때 참고한 문서는 Knowledge로 사용하여 다음 단계로 진행한다.

In [6]:
#Phase 2-2 Knowledged Optimization

def knowledge_optimization(question, knowledge):
    context = {
        'question': question,
        'knowledge': knowledge
    }
    
    knowledgeoptimization = read_prompt('prompt/KnowledgeOptimization.txt').format(**context)
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": knowledgeoptimization}
        ]
    )
    answer = completion.choices[0].message.content
    
    return answer

In [7]:
def judgment(question, knowledge, answer):
    
    query_knowledge = f'''#Query-1#: {question}
#Knowledge-1#: {knowledge}
    '''
    
    context = {
        'question': question,
        'answer': answer,
        'query_knowledge': query_knowledge,
    }
    judgement = read_prompt('prompt/Judgment.txt').format(**context)
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": judgement}
        ]
    )
    answer = completion.choices[0].message.content
    
    
    return answer
    # if 'INCONCLUSIVE' in answer:
    #     print("Answer is inconclusive")
    #     return answer
    # elif 'INCORRECT' in answer:
    #     print("사실과 다른 할루시네이션")
    #     return answer
    # else:
    #     print('Phase 2 passed')
    #     print("문제없음")
    #     return answer
    

question = "보험금 청구 서류 접수 후 일반적인 경우 보험금 지급기한은 언제까지입니까?"
knowledge = "그 서류를 접수한 날부터 3영업일 이내에 보험금을 지급하거나 보험료의 납입을 면제합니다."
answer = "서류 접수한 날부터 3영업일 이내에 보험금을 지급해야 합니다."

judgment(question, knowledge, answer)

'The answer states that the insurance payment should be made within 3 business days from the date of document submission. The query asks for the general payment deadline after the submission of insurance claim documents, and the knowledge confirms that payment should be made within the same timeframe of 3 business days from the date of submission. Both the answer and the knowledge imply the same timeframe for payment. Therefore, the final judgment is CORRECT.'

In [8]:
def HallucinationDetection(question, knowledge, answer):
    #Phase 1
    #Determine whether the QA is nonfabrication hallucination
    Phase_answer = nonfabrication_hallucination_checking(question, answer)
    if 'NONE' in Phase_answer:
        print("Non-Fabrication Hallucination")
        
        return print(korean_translater(Phase_answer))
    else:
        print("Phase 1 passed")
    #Phase 2-1 Knowledged Optimization
    OptimizedKnowledge = knowledge_optimization(question, knowledge)
    
    #Phase 2-2 Judgment
    Judgment = judgment(question, OptimizedKnowledge, answer)
    if 'INCONCLUSIVE' in Judgment:
        print("Answer is inconclusive")

    elif 'INCORRECT' in Judgment:
        print("사실과 다른 할루시네이션")
        return print(korean_translater(Judgment))
    else:
        print('Phase 2 passed')
        print('할루시네이션 없음')
        
    

question = "무배당 1~5종수술특약에서 보험수익자를 별도로 지정하지 않았다면 보험수익자는 누가 됩니까?"
knowledge = "이 계약에서 계약자가 보험수익자를 지정하지 않은 때에는 보험수익자를 피보험자로 합니다."
answer = "계약자가 보험수익자가 됩니다."


HallucinationDetection(question, knowledge, answer)
    
    

Phase 1 passed
사실과 다른 할루시네이션
무배당 1-5수술특약에서 피보험자가 별도의 보험수익자를 지정하지 않은 경우 계약자가 보험수익자가 된다는 답변이 있습니다. 그러나 제공된 지식은 이와 상반되게 별도의 수익자를 지정하지 않은 경우 피보험자가 수익자가 된다고 명시하고 있습니다. 따라서 이러한 모순으로 인해 정답에 대한 최종 판단은 올바르지 않습니다.
